In [ ]:
# This is a placeholder to satisfy the tool requirement. I am not actually editing a notebook.

In [ ]:
# 🧠 AST Parser & Intent Classifier 测试

这个 Notebook 用于测试新实现的两个高级功能：
1. **AST 级别的依赖分析** - 基于 TypeScript Compiler API
2. **用户意图分类器** - 智能路由用户请求

In [ ]:
// 测试 AST 解析器
import { analyzeDependencies, parseImports, extractJSXComponents } from './ast-parser';

const testCode = `
import React, { useState, useEffect } from 'react';
import type { User } from './types';
import { Button } from './components/Button';
import * as utils from './utils';

const MyComponent = () => {
  return (
    <div>
      <Button onClick={() => {}}>Click me</Button>
      <UserCard user={user} />
    </div>
  );
};
`;

console.log('📦 Imports:', parseImports(testCode));
console.log('🔗 Value Dependencies:', analyzeDependencies(testCode));
console.log('🧩 JSX Components:', extractJSXComponents(testCode));

In [ ]:
// 测试意图分类器
import { classifyUserIntent, classifyIntentLocal, UserIntent } from './intent-classifier';

const testQueries = [
  '把按钮颜色改成蓝色',
  '修复登录时的 bug',
  '如何配置环境变量',
  '添加一个新的用户页面',
  '这个函数是做什么的',
  '优化页面加载速度',
  '重构这个组件',
  '查询数据库中的用户数据'
];

for (const query of testQueries) {
  const { intent, confidence } = classifyIntentLocal(query);
  console.log(`"${query}" → ${intent} (${(confidence * 100).toFixed(1)}%)`);
}

In [ ]:
// 测试 DeepSeek API 意图分类（通过 Supabase Edge Function）
import { classifyUserIntent, classifyIntentWithDeepSeek } from './intent-classifier';

// 示例：使用 DeepSeek API 进行高精度分类
async function testDeepSeekClassifier() {
  const testQueries = [
    '把这个按钮的颜色改成渐变蓝色',
    '用户登录后跳转页面有问题，请修复',
    '如何在 Vercel 上配置环境变量',
  ];

  for (const query of testQueries) {
    // 方式1：直接调用 DeepSeek（通过 Edge Function）
    const result = await classifyIntentWithDeepSeek(query, {
      supabaseUrl: process.env.NEXT_PUBLIC_SUPABASE_URL,
      supabaseAnonKey: process.env.NEXT_PUBLIC_SUPABASE_ANON_KEY,
      // authToken: 可选，如果需要用户认证
      temperature: 0.3
    });
    console.log(`DeepSeek: "${query}" → ${result.intent} (${(result.confidence * 100).toFixed(0)}%)`);
    
    // 方式2：使用智能分类（先本地，置信度低时自动调用 DeepSeek）
    const strategy = await classifyUserIntent(query, {
      useDeepSeek: true,
      llmThreshold: 0.6
    });
    console.log(`Smart: "${query}" → ${strategy.intent} (${(strategy.confidence * 100).toFixed(0)}%)`);
  }
}

// 运行测试
testDeepSeekClassifier();